#제품 이상여부 판별 프로젝트

##1. 데이터 불러오기

###필수 라이브러리

In [1]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 77.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of selenium to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of selenium to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# 그래프 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

###데이터 읽어오기

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/Data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_edit_fs_corr_30.csv"),encoding='latin-1')
train_data

,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,...,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1,Dispense Volume(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1,HEAD Standby Position X Collect Result_Fill1,Machine Tact time Collect Result_Fill1,CURE END POSITION Z Collect Result_Fill2,target
0,16,14.9,8.4,14.7,1.04,0.58,1.02,274.51,274.51,66.00,...,3.6,3.42,838.4,458.7,244.520,244.520,244.520,128.0,33,Normal
1,10,21.3,4.9,21.3,1.49,0.34,1.49,282.15,282.15,130.85,...,3.6,3.42,458.5,157.0,244.400,244.400,289.000,56.5,33,Normal
2,16,14.7,8.5,14.7,1.61,0.93,1.61,274.33,274.33,66.00,...,3.5,3.32,837.9,458.1,244.415,244.415,244.415,128.0,33,Normal
3,10,21.3,8.4,21.3,1.49,0.58,1.49,282.15,282.15,130.85,...,3.6,3.42,458.8,157.0,244.300,244.300,289.000,57.7,33,Normal
4,10,9.7,4.9,9.6,0.67,0.34,0.67,284.80,284.80,133.50,...,3.6,3.42,458.5,157.0,244.400,244.400,289.000,89.9,32,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,10,17.0,5.0,17.0,1.19,0.35,1.19,282.15,282.15,130.85,...,3.6,3.42,458.5,157.0,244.400,244.400,289.000,51.8,33,Normal
40502,16,14.9,8.5,14.7,1.04,0.59,1.02,274.33,274.33,66.00,...,3.6,3.42,838.1,458.7,244.535,244.535,244.535,128.0,33,Normal
40503,16,14.2,8.2,14.3,0.99,0.57,1.00,274.78,274.78,66.00,...,3.6,3.42,838.4,458.4,244.543,244.543,244.543,128.0,33,Normal
40504,10,9.7,4.9,9.7,0.67,0.34,0.67,282.15,282.15,130.85,...,3.6,3.42,458.8,157.0,244.400,244.400,289.000,57.4,33,Normal


In [6]:
columns_df = train_data.columns
pd.set_option('display.max_seq_items', None)
columns_df

Index(['DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
       'Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
       'Head Clean Position Z Collect Result_Dam',
       'Head Purge Position Z Collect Result_Dam',
       'Head Zero Position X Collect Result_Dam',
       'Head Zero Position Z Collect Result_Dam',
       'Stage1 Line2 Distance Speed Collect Result_Dam',
       'Stage1 Line4 Distance Speed Collect Result_Dam',
       'Stage2 Circle1 Distance Speed Collect Result_Dam',
       'Stage2 Line2 Distance Speed Collect Result_Dam',
       'Stage2 Line3 Distance Spee

##오버샘플링

In [7]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

# SMOTE 객체 생성
smote = SMOTE(random_state=42)

# 타깃 열에서 'AbNormal' 값을 'AbNormal'로 유지하고, 나머지 값을 'Normal'로 수정
y_train = train_data['target'].apply(lambda x: 'AbNormal' if x == 'AbNormal' else 'Normal')

# X_train은 타깃 열을 제외한 나머지 열로 구성
X_train = train_data.drop(columns='target')

# 숫자가 아닌 열을 찾아서 레이블 인코딩 적용
label_encoder = LabelEncoder()
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = label_encoder.fit_transform(X_train[col])

smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_train_tomek, y_train_tomek = smoteto.fit_resample(X_train, y_train)

# 오버샘플링된_데이터 합치기
df_concat = pd.concat([X_train_tomek, y_train_tomek], axis=1)
df_concat

#count
df_normal = df_concat[df_concat["target"] == "Normal"]
df_abnormal = df_concat[df_concat["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

  Total: Normal: 38156, AbNormal: 38146


###데이터 분할

In [11]:
from pycaret.classification import *

exp_clf = setup(data = df_concat, target = 'target', session_id=123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(76302, 31)"
5,Transformed data shape,"(76302, 31)"
6,Transformed train set shape,"(53411, 31)"
7,Transformed test set shape,"(22891, 31)"
8,Numeric features,30
9,Preprocess,True


##모델비교

In [12]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7726,0.8701,0.7726,0.7743,0.7722,0.5451,0.5469,4.2160
et,Extra Trees Classifier,0.7715,0.8696,0.7715,0.7730,0.7712,0.5430,0.5445,3.9260
dt,Decision Tree Classifier,0.7702,0.8662,0.7702,0.7716,0.7699,0.5404,0.5418,0.4360
catboost,CatBoost Classifier,0.7612,0.8622,0.7612,0.7643,0.7605,0.5224,0.5255,23.9920
xgboost,Extreme Gradient Boosting,0.7505,0.8459,0.7505,0.7525,0.7500,0.5010,0.5030,1.1410
lightgbm,Light Gradient Boosting Machine,0.7442,0.8378,0.7442,0.7472,0.7434,0.4883,0.4914,2.7640
knn,K Neighbors Classifier,0.7095,0.7987,0.7095,0.7152,0.7075,0.4189,0.4246,3.1890
gbc,Gradient Boosting Classifier,0.6798,0.7525,0.6798,0.6817,0.6790,0.3596,0.3615,7.4000
ada,Ada Boost Classifier,0.6341,0.6851,0.6341,0.6349,0.6336,0.2682,0.2690,2.0410
ridge,Ridge Classifier,0.5801,0.6140,0.5801,0.5806,0.5795,0.1602,0.1607,0.2690


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

##모델 적합

In [13]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7776,0.8723,0.7776,0.7796,0.7772,0.5552,0.5572
1,0.7721,0.8663,0.7721,0.7743,0.7717,0.5443,0.5465
2,0.7764,0.8704,0.7764,0.7787,0.7760,0.5529,0.5552
3,0.7721,0.8709,0.7721,0.7739,0.7718,0.5443,0.5460
4,0.7660,0.8702,0.7660,0.7677,0.7656,0.5319,0.5337
5,0.7802,0.8715,0.7802,0.7811,0.7800,0.5604,0.5612
6,0.7675,0.8666,0.7675,0.7682,0.7673,0.5349,0.5357
7,0.7753,0.8734,0.7753,0.7779,0.7748,0.5506,0.5532
8,0.7665,0.8679,0.7665,0.7680,0.7662,0.5330,0.5346


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

##튜닝

In [14]:
tuned_rf = tune_model(rf, optimize = 'F1')
tuned_rf

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6146,0.6566,0.6146,0.6259,0.6057,0.2291,0.2402
1,0.6272,0.6750,0.6272,0.6423,0.6170,0.2544,0.2691
2,0.6188,0.6576,0.6188,0.6284,0.6115,0.2376,0.2470
3,0.6104,0.6666,0.6104,0.6179,0.6040,0.2207,0.2282
4,0.6111,0.6554,0.6111,0.6198,0.6039,0.2222,0.2308
5,0.6289,0.6760,0.6289,0.6365,0.6236,0.2578,0.2653
6,0.6122,0.6585,0.6122,0.6213,0.6049,0.2245,0.2333
7,0.6130,0.6543,0.6130,0.6213,0.6062,0.2259,0.2341
8,0.6231,0.6614,0.6231,0.6298,0.6182,0.2462,0.2528


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('  ').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [15]:
et = create_model('et')
tuned_et = tune_model(et, optimize = 'F1')
tuned_et

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7769,0.8709,0.7769,0.7785,0.7765,0.5537,0.5553
1,0.7738,0.8675,0.7738,0.7755,0.7735,0.5476,0.5493
2,0.7761,0.8716,0.7761,0.7779,0.7757,0.5521,0.5539
3,0.7703,0.8693,0.7703,0.7719,0.7699,0.5405,0.5422
4,0.7641,0.8699,0.7641,0.7656,0.7637,0.5282,0.5297
5,0.7764,0.8704,0.7764,0.7771,0.7763,0.5529,0.5535
6,0.7652,0.8645,0.7652,0.7658,0.7651,0.5304,0.5310
7,0.7748,0.8736,0.7748,0.7770,0.7743,0.5495,0.5517
8,0.7671,0.8690,0.7671,0.7682,0.7668,0.5342,0.5353


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6202,0.6648,0.6202,0.6258,0.6159,0.2404,0.2459
1,0.6345,0.6775,0.6345,0.6393,0.6314,0.2690,0.2738
2,0.6186,0.6595,0.6186,0.6223,0.6157,0.2372,0.2408
3,0.6248,0.6704,0.6248,0.6286,0.6220,0.2496,0.2533
4,0.6109,0.6640,0.6109,0.6166,0.6062,0.2218,0.2274
5,0.6321,0.6783,0.6321,0.6362,0.6293,0.2642,0.2683
6,0.6182,0.6639,0.6182,0.6210,0.6160,0.2365,0.2392
7,0.6239,0.6646,0.6239,0.6272,0.6214,0.2477,0.2510
8,0.6252,0.6659,0.6252,0.6281,0.6230,0.2503,0.2533


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)

##blending

In [16]:
# 방법 1
et = create_model('et')
rf = create_model('rf')

blender_2 = blend_models(estimator_list = [et, rf])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7769,0.8709,0.7769,0.7785,0.7765,0.5537,0.5553
1,0.7738,0.8675,0.7738,0.7755,0.7735,0.5476,0.5493
2,0.7761,0.8716,0.7761,0.7779,0.7757,0.5521,0.5539
3,0.7703,0.8693,0.7703,0.7719,0.7699,0.5405,0.5422
4,0.7641,0.8699,0.7641,0.7656,0.7637,0.5282,0.5297
5,0.7764,0.8704,0.7764,0.7771,0.7763,0.5529,0.5535
6,0.7652,0.8645,0.7652,0.7658,0.7651,0.5304,0.5310
7,0.7748,0.8736,0.7748,0.7770,0.7743,0.5495,0.5517
8,0.7671,0.8690,0.7671,0.7682,0.7668,0.5342,0.5353


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7776,0.8723,0.7776,0.7796,0.7772,0.5552,0.5572
1,0.7721,0.8663,0.7721,0.7743,0.7717,0.5443,0.5465
2,0.7764,0.8704,0.7764,0.7787,0.7760,0.5529,0.5552
3,0.7721,0.8709,0.7721,0.7739,0.7718,0.5443,0.5460
4,0.7660,0.8702,0.7660,0.7677,0.7656,0.5319,0.5337
5,0.7802,0.8715,0.7802,0.7811,0.7800,0.5604,0.5612
6,0.7675,0.8666,0.7675,0.7682,0.7673,0.5349,0.5357
7,0.7753,0.8734,0.7753,0.7779,0.7748,0.5506,0.5532
8,0.7665,0.8679,0.7665,0.7680,0.7662,0.5330,0.5346


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7787,0.8727,0.7787,0.7806,0.7784,0.5575,0.5593
1,0.7729,0.8676,0.7729,0.7751,0.7724,0.5458,0.5479
2,0.7779,0.8717,0.7779,0.7801,0.7775,0.5559,0.5581
3,0.7720,0.8710,0.7720,0.7737,0.7716,0.5439,0.5457
4,0.7658,0.8707,0.7658,0.7676,0.7654,0.5315,0.5333
5,0.7787,0.8718,0.7787,0.7796,0.7785,0.5574,0.5583
6,0.7669,0.8664,0.7669,0.7676,0.7667,0.5338,0.5345
7,0.7759,0.8742,0.7759,0.7784,0.7754,0.5518,0.5543
8,0.7682,0.8693,0.7682,0.7696,0.7679,0.5364,0.5378


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
# 방법 2
best_model_5 = compare_models(n_select=5)
blender_5 = blend_models(best_model_5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7726,0.8701,0.7726,0.7743,0.7722,0.5451,0.5469,3.9440
et,Extra Trees Classifier,0.7715,0.8696,0.7715,0.7730,0.7712,0.5430,0.5445,3.6000
dt,Decision Tree Classifier,0.7702,0.8662,0.7702,0.7716,0.7699,0.5404,0.5418,0.6690
catboost,CatBoost Classifier,0.7612,0.8622,0.7612,0.7643,0.7605,0.5224,0.5255,28.4980
xgboost,Extreme Gradient Boosting,0.7505,0.8459,0.7505,0.7525,0.7500,0.5010,0.5030,1.0820
lightgbm,Light Gradient Boosting Machine,0.7442,0.8378,0.7442,0.7472,0.7434,0.4883,0.4914,3.1100
knn,K Neighbors Classifier,0.7095,0.7987,0.7095,0.7152,0.7075,0.4189,0.4246,3.3390
gbc,Gradient Boosting Classifier,0.6798,0.7525,0.6798,0.6817,0.6790,0.3596,0.3615,7.2690
ada,Ada Boost Classifier,0.6341,0.6851,0.6341,0.6349,0.6336,0.2682,0.2690,2.1430
ridge,Ridge Classifier,0.5801,0.6140,0.5801,0.5806,0.5795,0.1602,0.1607,0.3690


Processing:   0%|          | 0/73 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7774,0.8764,0.7774,0.7797,0.7770,0.5548,0.5571
1,0.7744,0.8697,0.7744,0.7771,0.7738,0.5488,0.5514
2,0.7779,0.8756,0.7779,0.7798,0.7776,0.5559,0.5578
3,0.7738,0.8738,0.7738,0.7754,0.7735,0.5476,0.5492
4,0.7661,0.8736,0.7661,0.7680,0.7657,0.5323,0.5342
5,0.7817,0.8749,0.7817,0.7825,0.7815,0.5634,0.5641
6,0.7686,0.8687,0.7686,0.7697,0.7683,0.5372,0.5383
7,0.7785,0.8783,0.7785,0.7813,0.7780,0.5570,0.5598
8,0.7727,0.8732,0.7727,0.7743,0.7724,0.5454,0.5470


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
# 방법 3
blender_3= blend_models(estimator_list= [tuned_rf, tuned_et])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7787,0.8727,0.7787,0.7806,0.7784,0.5575,0.5593
1,0.7729,0.8676,0.7729,0.7751,0.7724,0.5458,0.5479
2,0.7779,0.8717,0.7779,0.7801,0.7775,0.5559,0.5581
3,0.7720,0.8710,0.7720,0.7737,0.7716,0.5439,0.5457
4,0.7658,0.8707,0.7658,0.7676,0.7654,0.5315,0.5333
5,0.7787,0.8718,0.7787,0.7796,0.7785,0.5574,0.5583
6,0.7669,0.8664,0.7669,0.7676,0.7667,0.5338,0.5345
7,0.7759,0.8742,0.7759,0.7784,0.7754,0.5518,0.5543
8,0.7682,0.8693,0.7682,0.7696,0.7679,0.5364,0.5378


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

#3. 모델 학습

###모델 학습

In [31]:
final_model = finalize_model(blender_3)

#4. 제출하기

###테스트 데이터 예측

테스트 데이터 불러오기

In [32]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"),encoding='latin-1')

In [33]:
test_pred = predict_model(final_model, data = test_data)
test_pred

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,...,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,prediction_label,prediction_score
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000,12.5,...,270.000000,50,85,19.799999,13.000000,195,1,0,Normal,0.5165
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000,12.5,...,91.800003,270,50,85.000000,19.799999,14,256,1,AbNormal,0.6857
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240,2.5,...,270.000000,50,85,19.700001,1.000000,98,1,0,Normal,0.7224
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000,12.5,...,91.800003,270,50,85.000000,20.000000,14,0,1,Normal,0.9330
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240,2.5,...,91.800003,270,50,85.000000,19.799999,1,215,1,AbNormal,0.6967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000,12.5,...,91.800003,270,50,85.000000,19.500000,14,131,1,Normal,0.5713
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000,12.5,...,91.800003,270,50,85.000000,19.799999,12,279,1,Normal,0.9546
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240,2.5,...,91.800003,270,50,85.000000,20.500000,4,66,1,Normal,0.8256
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240,2.5,...,270.000000,50,85,18.900000,1.000000,117,1,0,AbNormal,0.5788


###제출 파일 작성

In [35]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv(os.path.join(ROOT_DIR,"submission.csv"),encoding='latin-1')

# 'test_pred'의 첫 번째 컬럼 값을 'df_sub'의 'target' 컬럼에 할당합니다.
df_sub["target"] = test_pred["prediction_label"]

# 제출 파일 저장
df_sub.to_csv("submission_0826_belender3.csv", index=False)